In [268]:
import json
import os
import cv2
import numpy as np
import pandas as pd

from PIL import Image
from PIL import ImageDraw

def get_json_data(file_path):
    with open(file_path) as data:
        json_data = json.load(data)
    return json_data



def get_images_info_from_COCO_data(COCO_data):
    return COCO_data['images']


def get_image_name_and_id_from_json(coco_images_info):
    dict_keys   = coco_images_info[0].keys()
    df          = pd.DataFrame(coco_images_info)
    del_columns = [column for column in dict_keys if column not in ['file_name', 'id']]
    df.drop(del_columns, axis = 1, inplace = True)
    df.columns  = ['image_name', 'image_id']
    return df



def get_categories_df(coco_categories_info):
    coco_categories_df = pd.DataFrame(coco_categories_info)
    coco_categories_df = coco_categories_df.set_index('id')
    return coco_categories_df



def get_annotations_df(coco_annotations_info, images_df):
    annotations_df = pd.DataFrame(coco_annotations_info)
#     images_annotations_df = pd.merge(coco_annotations_ddf, images_df, on = 'image_id')
#     images_annotations_df.drop(['bbox', 'area'], axis = 1, inplace = True)
    return annotations_df

In [4]:
COCO_data         = get_json_data('/Users/vijay/Downloads/annotations/instances_train2014.json')
coco_images_info  = get_images_info_from_COCO_data(COCO_data)
images_name_id_df = get_image_name_and_id_from_json(coco_images_info)
categories_df     = get_categories_df(COCO_data['categories'])




In [216]:


def read_coco_images(dir_path):
    images_arr   = {}
    images_names = os.listdir(dir_path)
    images_arr   = read_image_(dir_path, images_names, images_arr)
        
    return pd.DataFrame(images_arr.items(), columns = ['image_name', 'pixel_data'])

    
    
##########################################################    

def read_image_(path, image_names, images_arr):
    
    try:
        ii = 0
        for image_name in image_names:
            if image_name.split(".")[-1].lower() in {"jpeg", "jpg", "png"}:

                image     = cv2.cvtColor(cv2.imread(path + "/" + image_name), cv2.COLOR_RGB2GRAY)
                image     = image / 255  
                images_arr[str(image_name)] = image            
        return images_arr
    
    except Exception as e:
        print(str(e))
        
        
def merge_images_id_and_data_dfs(images_df, images_name_id_df):
    images_names_df = pd.merge(images_df, images_name_id_df, on = 'image_name')
    return images_names_df

In [269]:
images_df = read_coco_images('/Users/vijay/Downloads/Datasets/COCO_sample/')
images_names_df = merge_images_id_and_data_dfs(images_df, images_name_id_df)
annotations_df = get_annotations_df(COCO_data['annotations'], images_names_df)

In [378]:
import math
from matplotlib.path import Path
import matplotlib.patches as patches
import pylab


########################################################## 

def get_image_id_from_name(image_name, images_names_df):
    return_id = 0
    for _, row in images_names_df.iterrows():
        if row['image_name'] == image_name:
            return_id = row['image_id']
            break
    return return_id



########################################################## 

def get_indices_having_same_image_id(image_id, annotations_df):
    indices_list = np.where(annotations_df['image_id'] == image_id)[0]
    return indices_list



########################################################## 

def get_image_from_image_name(image_name, images_df):
    image = images_df.loc[images_df['image_name'] == image_name]['pixel_data']
    return image.iloc[0]



########################################################## 

def get_super_categories_as_one_hot_labels(categories_df):
    
    try:
        '''converting super categories into one-hot labels'''
        super_categories = list(set(categories_df['supercategory']))
        category_index = 1
        supercategory_dict = {}
        for category in super_categories:
            supercategory_dict[category] = category_index
            category_index = category_index + 1

        return supercategory_dict
    except Exception as e:
        print('error is - ' + str(e))
        print('error in get_super_categories_as_one_hot_labels')



########################################################## 

def get_super_category(category_id, categories_df, supercategory_dict):
    try:
        super_category = categories_df.loc[category_id]['supercategory']
        return supercategory_dict[super_category]
    except Exception as e:
        print('error is - ' + str(e))
        print('error in get_super_category')



########################################################## 

def get_all_segmentations_for_each_image_id(image_id, annotations_df, categories_df, supercategory_dict):
    
    try:
        segmentation_list     = []
        indices_with_image_id = get_indices_having_same_image_id(image_id, annotations_df)
        for index in indices:
            row              = annotations_df.loc[index]
            spr_category_val = get_super_category(row['category_id'], categories_df, supercategory_dict)
            segmentation_list.append((row['segmentation'], spr_category_val))

        return segmentation_list
    except Exception as e:
        print('error is - ' + str(e))
        print('error in get_all_segmentations_for_each_image_id')
    


########################################################## 

def convert_segmentation_vals_as_xy(segmentation_li):
    try:
        len_seg   = np.shape(segmentation_li)[0]
        xy = [[segmentation[index], segmentation[index + 1]] for index in range(0, len_seg, 2)]
        return xy
    except Exception as e:
        print('error in convert_segmentation_vals_as_xy')

        

##########################################################


def fill_image_with_category_value(image_arr, category_value):
    try:
        img = Image.fromarray(image_arr.astype(np.uint8))
        draw = ImageDraw.Draw(img)
        draw.polygon([tuple(p) for p in vertices], fill = category_value)
        return np.asarray(img)
    except Exception as e:
        print('error is - ' + str(e))
        print('error in fill_image_with_category_value')
    
    


########################################################## 

def label_the_images_with_super_categories(image_shape, segmentation_list):
    
    try:
        '''Segmentation in annotations is a list of lists
        --- in the function get_all_segmentations_for_each_image_id, we collected all the segmentation
        related to one image_id and then grouped them into another list.
        --- Thereore, segmentation_list in this method is a list of list of lists'''
        image_label_array = np.zeros(image_shape)
        for segmenation in segementation_list:
            xy = []
            for sub_segementation in segmentation:
                xy = xy + convert_segmentation_vals_as_xy(sub_segmentation[0])

            image_label_array = fill_image_with_category_value(image_label_arr, sub_segmentation[1])

        return image_label_array
    except Exception as e:
        print('error is - ' + str(e))
        print('error in label_the_images_with_super_categories')
            
            
            
            
########################################################### 
def start_labelling_the_images(images_df, images_names_df, annotations_df, categories_df):
    try:
        img_labels_df    = {}
        images_names       = np.array(images_df['image_name'])
        supercategory_dict = get_super_categories_as_one_hot_labels(categories_df)
        for image_name in images_names:
            image_id      = get_image_id_from_name(image_name, images_names_df)
            image         = get_image_from_image_name(image_name, images_df)
            segments_list = get_all_segmentations_for_each_image_id(image_id, annotations_df, categories_df,
                                                                               supercategory_dict)
            label_array   = label_the_images_with_super_categories(np.shape(image), segments_list)

            img_labels_df[image_name] = label_array

        return pd.DataFrame(img_labels.items(), columns = ['image_name', 'label_data'])
    
    except Exception as e:
        print('error is - ' + str(e))
        print('error in start_labelling_the_images')
        
    
    